## Step1. 탐색: 시간정보가 포함된 데이터 살펴보기

### market price 데이터셋의 기본 정보

In [5]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

file_path = 'C:/Users/yubin/Documents/GitHub/data_analysis/market-price.csv'
bitcoin_df = pd.read_csv(file_path, names = ['day', 'price'])

In [6]:
# 기본 정보 출력
print(bitcoin_df.shape)
print(bitcoin_df.info())

(367, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 2 columns):
day      367 non-null object
price    367 non-null object
dtypes: object(2)
memory usage: 5.8+ KB
None


In [7]:
bitcoin_df.tail()

,day,price
362,2020-12-26 00:00:00,24693.58
363,2020-12-27 00:00:00,26443.21
364,2020-12-28 00:00:00,26246.58
365,2020-12-29 00:00:00,27036.69
366,2020-12-30 00:00:00,27376.37


### 시계열 데이터 살펴보기

In [12]:
# to_datetime으로 day 피처를 시계열 피처로 변환
bitcoin_df['day'] = pd.to_datetime(bitcoin_df['day'])

# day 데이터프레임의 index로 설정
bitcoin_df.index = bitcoin_df['day']
bitcoin_df.set_index('day', inplace=True)
bitcoin_df.head()

ValueError: ('Unknown string format:', 'Timestamp')

In [10]:
bitcoin_df.describe()

,day,price
count,367,367
unique,367,367
top,2020-05-17 00:00:00,26443.21
freq,1,1


In [11]:
# 일자별 비트코인 시세 시각화
bitcoin_df.plot()
plt.show()

TypeError: Empty 'DataFrame': no numeric data to plot

## Step2. 예측: 파이썬 라이브러리를 활용한 시세 예측

### ARIMA 모델 활용하기: 모델 학습

In [13]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델 학습
model = ARIMA(bitcoin_df.price.values, order=(2,1,2))
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### 모델의 성능 & 예측 결과 시각화
#### 학습한 모델이 학습 데이터에 대해서 예측한 결과만을 시각화

In [14]:
fig = model_fit.plot_predict() # 학습 데이터에 대한 예측 결과 (첫번째 그래프)
residuals = pd.DataFrame(model_fit.resid) # 잔차의 변동을 시각화 (두번째 그래프)
residuals.plot()

NameError: name 'model_fit' is not defined

### 실제 데이터와의 비교

In [15]:
forecast_data = model_fit.forecast(steps=5) # 학습 데이터셋으로부터 5일 뒤 예측
# 테스트 데이터셋 불러오기
test_file_path = '../data/market-price-test.csv'
bitcoin_test_df = pd.read_csv(test_file_path, names=['ds', 'y'])

pred_y = forecast_data[0].tolist() # 마지막 5일의 예측 데이터 (2018-08-27 ~ 2018-08-31)
test_y = bitcoin_test_df.y.values # 실제 5일 가격 데이터. (2018-08-27 ~ 2018-08-31)
pred_y_lower = [] # 마지막 5일의 예측 데이터의 최소값
pred_y_upper = [] # 마지막 5일의 예측 데이터의 최대값
for lower_upper in forecast_data[2]:
    lower = lower_upper[0]
    upper = lower_upper[1]
    pred_y_lower.append(lower)
    pred_y_upper.append(upper)

NameError: name 'model_fit' is not defined

In [16]:
plt.plot(pred_y, color="gold") # 모델이 예상한 가격 그래프
plt.plot(pred_y_lower, color="red") # 모델이 예상한 최소가격 그래프
plt.plot(pred_y_upper, color="blue") # 모델이 예상한 최대가격 그래프
plt.plot(test_y, color="green") # 실제 가격 그래프

NameError: name 'pred_y' is not defined

In [17]:
plt.plot(pred_y, color="gold") # 모델이 예상한 가격 그래프
plt.plot(test_y, color="green") # 실제 가격 그래프

NameError: name 'pred_y' is not defined

In [18]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

rmse = sqrt(mean_squared_error(pred_y, test_y))
print(rmse)

NameError: name 'pred_y' is not defined

### Facebook Prophet 활용하기

In [19]:
from fbprophet import Prophet

# prophet을 사용하기 위해서는 다음과 같이 피처의 이름을 변경해야 함 : 'ds', 'y'
bitcoin_df = pd.read_csv(file_path, names=['ds', 'y'])
prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=True,
                  weekly_seasonality=True, daily_seasonality=True,
                  changepoint_prior_scale=0.5)
prophet.fit(bitcoin_df)

ModuleNotFoundError: No module named 'fbprophet'

In [20]:
#모델로부터 예측할 기간을 정의하고, 결과를 도출

# 5일을 내다보며 예측
future_data = prophet.make_future_dataframe(periods=5, freq='d')
forecast_data = prophet.predict(future_data)

NameError: name 'prophet' is not defined

In [21]:
forecast_data.tail(5)

NameError: name 'forecast_data' is not defined

In [22]:
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

NameError: name 'forecast_data' is not defined

#### 결과의 시각화
#### 전체 데이터를 기반으로 학습한, 5일 단위의 예측 결과를 시각화

In [23]:
fig1 = prophet.plot(forecast_data)

NameError: name 'prophet' is not defined

In [24]:
#seasonality_mode로 설정했었던 기간별 트렌드 정보를 시각화
fig2 = prophet.plot_components(forecast_data)

NameError: name 'prophet' is not defined

#### 모델의 성능

In [25]:
y = bitcoin_df.y.values[5:] # 첫 5일을 제외한 실제 가격 데이터
y_pred = forecast_data.yhat.values[5:-5] # 첫 5일, 마지막 5일을 제외한 예측 가격 데이터

AttributeError: 'DataFrame' object has no attribute 'y'

In [26]:
rmse = sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)
print(rmse)
print(r2)

NameError: name 'y' is not defined